In [ ]:
import functools
import os
import sys
import typing
import random
import joblib
import dcor
import math
from math import ceil, log, sqrt
import networkx as nx
import numpy as np
import pandas as pd
import pingouin as pg
from tqdm.auto import tqdm
import itertools
from itertools import combinations
from copy import deepcopy
import scipy.stats as stats
import scipy.special as special
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import Ridge, LinearRegression, LassoCV
from sklearn.metrics import adjusted_mutual_info_score
from collections import Counter, defaultdict
from econml.dml import CausalForestDML, LinearDML
from causallearn.graph.GraphNode import GraphNode
from causallearn.utils.PCUtils.BackgroundKnowledge import BackgroundKnowledge
from causallearn.search.ConstraintBased.PC import pc
from causallearn.search.ScoreBased.ExactSearch import bic_exact_search
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.search.PermutationBased.GRaSP import grasp
import semopy
from semopy import Model
from semopy.inspector import inspect

In [ ]:
def graph_nodes_representation(graph, nodelist):
    """
    Create an alternative representation of a graph which is hashable
    and equivalent graphs have the same hash.

    Python cannot PROPERLY use nx.Graph/DiGraph as key for
    dictionaries, because two equivalent graphs with just different
    order of the nodes would result in different keys. This is
    undesirable here.

    So here we transform the graph into an equivalent form that is
    based on a specific nodelist and that is hashable. In this way,
    two equivalent graphs, once transformed, will result in identical
    keys.

    So we use the following trick: extract the adjacency matrix
    (with nodes in a fixed order) and then make a hashable thing out
    of it, through tuple(array.flatten()):
    """

    # This get the adjacency matrix with nodes in a given order, as
    # numpy array (which is not hashable):
    adjacency_matrix = nx.adjacency_matrix(graph, nodelist=nodelist).todense()

    # This transforms the numpy array into a hashable object:
    hashable = tuple(adjacency_matrix.flatten())

    return hashable

def create_graph_label():
    """
    Create a dictionary from graphs to labels, in two formats.
    """
    graph_label = {
        nx.DiGraph([("X", "Y"), ("v", "X"), ("v", "Y")]): "Confounder",
        nx.DiGraph([("X", "Y"), ("X", "v"), ("Y", "v")]): "Collider",
        nx.DiGraph([("X", "Y"), ("X", "v"), ("v", "Y")]): "Mediator",
        nx.DiGraph([("X", "Y"), ("v", "X")]):             "Cause of X",
        nx.DiGraph([("X", "Y"), ("v", "Y")]):             "Cause of Y",
        nx.DiGraph([("X", "Y"), ("X", "v")]):             "Consequence of X",
        nx.DiGraph([("X", "Y"), ("Y", "v")]):             "Consequence of Y",
        nx.DiGraph({"X": ["Y"], "v": []}):                "Independent",
    }

    nodelist = ["v", "X", "Y"]

    # This is an equivalent alternative to graph_label but in a form
    # for which two equivalent graphs have the same key:
    adjacency_label = {
        graph_nodes_representation(graph, nodelist): label
        for graph, label in graph_label.items()
    }

    return graph_label, adjacency_label

def get_labels(adjacency_matrix, adjacency_label):
    """
    Transform an adjacency_matrix (as pd.DataFrame) into a dictionary of variable:label
    """

    result = {}
    for variable in adjacency_matrix.columns.drop(["X", "Y"]):
        submatrix = adjacency_matrix.loc[[variable, "X", "Y"], [variable, "X", "Y"]]  # this is not hashable
        key = tuple(submatrix.values.flatten())  # this is hashable and a compatible with adjacency_label
    
        result[variable] = adjacency_label[key]

    return result

graph_label, adjacency_label = create_graph_label()

In [ ]:
def apply_mapping(df, mapping):
    df_new = df.copy()
    # 创建临时映射以避免冲突
    temp_mapping = {k: f'_temp_{k}' for k in mapping.keys()}
    df_new.rename(columns=temp_mapping, inplace=True)
    if df_new.shape[0] == df_new.shape[1]:  # 如果是方阵，如标签矩阵
        df_new.rename(index=temp_mapping, inplace=True)
    # 应用最终映射
    final_mapping = {f'_temp_{k}': v for k, v in mapping.items()}
    df_new.rename(columns=final_mapping, inplace=True)
    if df_new.shape[0] == df_new.shape[1]:
        df_new.rename(index=final_mapping, inplace=True)
    return df_new

def check_duplicate_columns(df):
    """检查是否存在重复的列名"""
    return df.columns.duplicated().any()

def augment_data(X_train, y_train):
    new_X_train = X_train.copy()
    new_y_train = y_train.copy()
    for sample_id in X_train.keys():
        X = X_train[sample_id]
        y = y_train[sample_id]
        variables = list(X.columns)
        dim = len(variables)
        # 提取因果关系对
        edges = []
        for u in y.index:
            for v in y.columns:
                if y.loc[u, v] == 1:
                    edges.append((u, v))
        # 排除涉及 X 和 Y 的边
        edges_no_XY = [(u, v) for (u, v) in edges if u not in ['X', 'Y'] and v not in ['X', 'Y']]
        if dim >= 4:
            edges_to_use = edges_no_XY
            attempts = 0
            success = False
            while attempts < 3 and not success:
                if not edges_to_use:
                    break  # 没有合适的边，跳出循环
                u, v = random.choice(edges_to_use)
                mapping = {'X': u, 'Y': v, u: 'X', v: 'Y'}
                # 应用映射到特征矩阵和标签矩阵
                X_new = apply_mapping(X, mapping)
                y_new = apply_mapping(y, mapping)
                # 检查特征矩阵是否有重复列
                if check_duplicate_columns(X_new):
                    attempts += 1
                    continue  # 重试
                else:
                    # 没有重复列，存储新的数据
                    new_sample_id = '0' + sample_id
                    new_X_train[new_sample_id] = X_new
                    new_y_train[new_sample_id] = y_new
                    success = True
            if not success:
                # 没有找到合适的映射，复制原始数据
                new_sample_id = '0' + sample_id
                new_X_train[new_sample_id] = X.copy()
                new_y_train[new_sample_id] = y.copy()
        else:
            # 对于维度较低的数据，允许涉及 X 和 Y 的边
            edges_to_use = edges
            if not edges_to_use:
                # 没有边，复制原始数据
                new_sample_id = '0' + sample_id
                new_X_train[new_sample_id] = X.copy()
                new_y_train[new_sample_id] = y.copy()
                continue
            u, v = random.choice(edges_to_use)
            mapping = {'X': u, 'Y': v, u: 'X', v: 'Y'}
            # 应用映射到特征矩阵和标签矩阵
            X_new = apply_mapping(X, mapping)
            y_new = apply_mapping(y, mapping)
            # 检查特征矩阵是否有重复列
            if check_duplicate_columns(X_new):
                # 如果有重复列，复制原始数据
                new_sample_id = '0' + sample_id
                new_X_train[new_sample_id] = X.copy()
                new_y_train[new_sample_id] = y.copy()
            else:
                # 没有重复列，存储新的数据
                new_sample_id = '0' + sample_id
                new_X_train[new_sample_id] = X_new
                new_y_train[new_sample_id] = y_new
    return new_X_train, new_y_train

In [ ]:
X_train = pd.read_pickle('./data/X_train.pickle')
y_train = pd.read_pickle('./data/y_train.pickle')
## 需要修改为增强后的数据

In [ ]:
"""离散化统计指标的工具函数"""
def discretize_sequence(x, ffactor=10):
    """
    将连续序列离散化。

    参数:
    x (array-like): 输入的连续变量，范围在 [-1, 1] 之间。
    ffactor (int): 离散化因子，用于缩放和离散化。

    返回:
    np.ndarray: 离散化后的序列（整数类型）。
    """
    # 确保输入在 [-1, 1] 范围内
    x = np.clip(x, -1, 1)
    # 缩放并四舍五入
    x = np.round(x * ffactor).astype(int)
    return x

def compute_discrete_probability(x):
    """
    计算离散概率分布。

    参数:
    x (iterable): 输入的离散化后的序列，可以是单变量或联合变量。

    返回:
    Counter: 元素及其计数。
    """
    return Counter(x)

def discrete_entropy(x, bias_factor=0.7):
    """
    计算离散熵。

    参数:
    x (iterable): 输入的离散化后的序列，可以是单变量或联合变量。
    bias_factor (float): 偏差因子，用于修正有限样本的熵估计。

    返回:
    float: 计算得到的熵值。
    """
    c = compute_discrete_probability(x)
    pk = np.array(list(c.values()), dtype=float)
    pk_sum = pk.sum()
    if pk_sum == 0:
        return 0.0
    pk /= pk_sum
    # 避免 log(0) 问题，添加一个很小的常数
    vec = pk * np.log(pk + 1e-12)
    S = -np.sum(vec)
    # 添加偏差项
    bias = bias_factor * (len(pk) - 1) / (2.0 * len(x))
    return S + bias

def discrete_joint_entropy(x, y):
    """
    计算两个离散序列的联合熵 H(X, Y)。

    参数:
    x, y (array-like): 输入的离散化后的序列。

    返回:
    float: 联合熵 H(X, Y)。
    """
    joint = list(zip(x, y))  # 将 x 和 y 配对
    return discrete_entropy(joint)

def normalized_error_probability(x, y):
    """
    计算归一化的错误概率。

    该函数通过构建联合概率矩阵，计算分类错误的概率，并将其归一化。

    参数:
    x, y (array-like): 输入的离散化后的序列。

    返回:
    float: 归一化的错误概率。
    """
    cx = Counter(x)
    cy = Counter(y)

    sorted_cx = sorted(cx.keys())
    sorted_cy = sorted(cy.keys())

    # 统计联合频数
    pxy = defaultdict(int)
    for a, b in zip(x, y):
        pxy[(a, b)] += 1

    total = sum(pxy.values())
    if total == 0:
        return 0.0  # 或者根据需求返回其他值

    # 构建联合概率矩阵
    pxy_matrix = np.array([
        [pxy.get((a, b), 0) for b in sorted_cy]
        for a in sorted_cx
    ], dtype=float)

    # 归一化为概率
    pxy_matrix /= total

    # 计算每行的最大概率
    max_per_row = pxy_matrix.max(axis=1)
    perr = 1 - np.sum(max_per_row)

    # 计算每列的概率和的最大值
    sum_per_column = pxy_matrix.sum(axis=0)
    max_perr = 1 - np.max(sum_per_column)

    # 归一化错误概率
    pnorm = perr / max_perr if max_perr > 0 else perr
    return pnorm

def discrete_divergence(cx, cy):
    """
    计算两个离散分布之间的KL散度（Kullback-Leibler Divergence）。

    KL散度衡量了分布 cx 相对于分布 cy 的差异，是信息论中的一个重要概念。

    参数:
    cx (Counter): 第一个离散分布的元素计数。
    cy (Counter): 第二个离散分布的元素计数。

    返回:
    float: KL散度 D_KL(cx || cy)。
    """
    # 创建 cy 的副本，避免修改原始对象
    cy = cy.copy()

    # 为了避免 cy 中某些元素的概率为零，将它们的计数设为 1
    for a in cx:
        if cy[a] == 0:
            cy[a] = 1

    # 计算概率
    nx = float(sum(cx.values()))
    ny = float(sum(cy.values()))

    kl_div = 0.0
    for a, v in cx.items():
        px = v / nx
        py = cy[a] / ny
        kl_div += px * np.log(px / py)
    return kl_div

def discrete_conditional_entropy(x, y):
    """
    计算两个离散序列的条件熵 H(X|Y)。

    条件熵衡量了在已知 Y 的情况下，X 的不确定性。

    参数:
    x, y (array-like): 输入的离散化后的序列。

    返回:
    float: 条件熵 H(X|Y)。
    """
    joint_entropy = discrete_joint_entropy(x, y)
    entropy_y = discrete_entropy(y)
    return joint_entropy - entropy_y

def adjusted_mutual_information_score(x, y):
    """
    计算两个离散序列的调整互信息（Adjusted Mutual Information, AMI）。

    AMI 是互信息的一种调整版本，考虑了随机期望的互信息，通常用于聚类评估。

    参数:
    x, y (array-like): 输入的离散化后的序列。

    返回:
    float: 调整后的互信息。
    """
    return adjusted_mutual_info_score(x, y)

def discrete_mutual_information(x, y):
    """
    计算两个离散序列的互信息 I(X; Y)。

    互信息衡量了两个变量之间共享的信息量，是信息论中的一个基本概念。

    参数:
    x, y (array-like): 输入的离散化后的序列。

    返回:
    float: 互信息 I(X; Y)。
    """
    entropy_x = discrete_entropy(x)
    entropy_y = discrete_entropy(y)
    joint_entropy = discrete_joint_entropy(x, y)
    mutual_info = entropy_x + entropy_y - joint_entropy
    # 避免由于数值误差导致的负值
    mutual_info = max(mutual_info, 0)
    return mutual_info

# -------------------
# Helper Functions
# -------------------

def normalize_discrete(x):
    """
    对离散化后的序列进行标准化处理。

    参数:
    x (array-like): 离散化后的x序列。

    返回:
    np.ndarray: 标准化后的x序列。
    """
    if len(set(x)) < 2:
        return np.zeros_like(x, dtype=float)
    x_mean = np.mean(x)
    x_std = np.std(x)
    if x_std > 0:
        return (x - x_mean) / x_std
    else:
        return x - x_mean

def to_numerical(x_discrete, y_continuous):
    """
    将类别型的离散x转换为数值型，通过将每个唯一的x值替换为对应y的平均值。

    参数:
    x_discrete (array-like): 离散化后的x数组。
    y_continuous (array-like): 与x对应的连续y数组。

    返回:
    np.ndarray: 数值型的x数组，每个x值被替换为对应的y平均值。
    """
    dx = defaultdict(lambda: [0.0, 0])
    for a, b in zip(x_discrete, y_continuous):
        dx[a][0] += b
        dx[a][1] += 1
    for a in dx:
        dx[a][0] /= dx[a][1] if dx[a][1] > 0 else 1e-12
    x_numerical = np.array([dx[a][0] for a in x_discrete], dtype=float)
    x_numerical = (x_numerical - np.mean(x_numerical)) / np.std(x_numerical) if np.std(x_numerical) > 0 else x_numerical
    return x_numerical

def count_unique(x):
    """
    计算数组中唯一元素的数量。

    参数:
    x (array-like): 输入数组。

    返回:
    int: 唯一元素的数量。
    """
    return len(set(x))

# -------------------
# Feature Engineering Functions
# -------------------

def normalized_entropy_baseline(x):
    """
    计算给定归一化x的标准化熵基线。

    参数:
    x (array-like): 离散且归一化的x序列。

    返回:
    float: 标准化熵基线值。
    """
    if len(set(x)) < 2:
        return 0.0
    xs = np.sort(x)
    delta = xs[1:] - xs[:-1]
    delta = delta[delta != 0]
    if len(delta) == 0:
        return 0.0
    hx = np.mean(np.log(delta))
    hx += special.psi(len(delta))
    hx -= special.psi(1)
    return hx

def normalized_entropy(x, m=2):
    """
    计算标准化熵。

    参数:
    x (array-like): 离散且归一化的x序列。
    m (int): delta计算的参数。

    返回:
    float: 标准化熵值。
    """
    cx = Counter(x)
    if len(cx) < 2:
        return 0.0
    xk = np.array(list(cx.keys()), dtype=float)
    xk.sort()
    if len(xk) < 2:
        return 0.0
    delta = (xk[1:] - xk[:-1]) / m
    counter = np.array([cx[i] for i in xk], dtype=float)
    hx = np.sum(counter[1:] * np.log(delta / counter[1:])) / len(x)
    hx += (special.psi(len(delta)) - np.log(len(delta)))
    hx += np.log(len(x))
    hx -= (special.psi(m) - np.log(m))
    return hx

def igci(x, y):
    """
    计算IGCI（信息几何因果推断）度量。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。

    返回:
    float: IGCI度量值。
    """
    # 检查是否有足够的唯一值
    if len(set(x)) < 2:
        return 0.0
    
    # 判断是否有重复的x值
    if len(x) != len(set(x)):
        dx = defaultdict(lambda: [0.0, 0])
        for a, b in zip(x, y):
            dx[a][0] += b
            dx[a][1] += 1
        for a in dx:
            dx[a][0] /= dx[a][1] if dx[a][1] > 0 else 1e-12
        # 构建联合序列
        xy = np.array([[a, dx[a][0]] for a in dx.keys()], dtype=float)
        # 获取每个x的计数
        counter = np.array([dx[a][1] for a in xy[:, 0]], dtype=float)
    else:
        # 如果x没有重复，直接排序
        xy = np.array(sorted(zip(x, y)), dtype=float)
        counter = np.ones(len(x))
    
    # 计算相邻差值
    delta = xy[1:] - xy[:-1]
    # 选择y差值不为0的样本
    selec = delta[:, 1] != 0
    delta = delta[selec]
    counter = np.minimum(counter[1:], counter[:-1])[selec]
    
    if len(delta) == 0:
        return 0.0
    
    # 添加一个极小值epsilon，避免log(0)
    epsilon = 1e-12
    ratio = (delta[:, 0] + epsilon) / np.abs(delta[:, 1])
    ratio = np.where(ratio > 0, ratio, epsilon)
    
    # 计算 hxy，避免返回 NaN
    with np.errstate(divide='ignore', invalid='ignore'):
        hxy = np.sum(counter * np.log(ratio)) / len(x)
    
    # 检查 hxy 是否为有效数值
    if np.isnan(hxy):
        return 0.0
    
    return hxy

def uniform_divergence(x, m=2):
    """
    计算统一散度。

    参数:
    x (array-like): 离散且归一化的x序列。
    m (int): delta计算的参数。

    返回:
    float: 统一散度值。
    """
    cx = Counter(x)
    xk = np.array(list(cx.keys()), dtype=float)
    xk.sort()
    delta = np.zeros(len(xk))
    if len(xk) > 1:
        delta[0] = xk[1] - xk[0]
        if len(xk) > m:
            delta[1:-1] = (xk[m:] - xk[:-m]) / m
        else:
            delta[1:-1] = (xk[-1] - xk[0]) / (len(xk) - 1)
        delta[-1] = xk[-1] - xk[-2]
    else:
        delta = np.array([np.sqrt(12)], dtype=float)  # 假设均匀分布在[-1,1]

    counter = np.array([cx[i] for i in xk], dtype=float)
    delta_sum = np.sum(delta)
    if delta_sum > 0:
        delta = delta / delta_sum
    else:
        delta = delta
    if len(xk) > 1:
        hx = np.sum(counter * np.log(counter / delta)) / len(x)
    else:
        hx = 0.0
    hx -= np.log(len(x))
    hx += (special.psi(m) - np.log(m))
    return hx

def normalized_skewness(x):
    """
    计算x的标准化偏度。

    参数:
    x (array-like): 离散且归一化的x序列。

    返回:
    float: 标准化偏度值。
    """
    return stats.skew(x)

def normalized_kurtosis(x):
    """
    计算x的标准化峰度。

    参数:
    x (array-like): 离散且归一化的x序列。

    返回:
    float: 标准化峰度值。
    """
    return stats.kurtosis(x)

def normalized_moment(x, y, n, m):
    """
    计算x和y的标准化联合矩。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。
    n (int): x的幂次。
    m (int): y的幂次。

    返回:
    float: 标准化的联合矩值。
    """
    return np.mean((x ** n) * (y ** m))

def moment21(x, y):
    """
    计算标准化联合矩 I(X^2 * Y)。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。

    返回:
    float: 联合矩 I(X^2 * Y)。
    """
    return normalized_moment(x, y, 2, 1)

def moment22(x, y):
    """
    计算标准化联合矩 I(X^2 * Y^2)。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。

    返回:
    float: 联合矩 I(X^2 * Y^2)。
    """
    return normalized_moment(x, y, 2, 2)

def moment31(x, y):
    """
    计算标准化联合矩 I(X^3 * Y)。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。

    返回:
    float: 联合矩 I(X^3 * Y)。
    """
    return normalized_moment(x, y, 3, 1)

def fit_pairwise(x, y):
    """
    拟合多项式到x和y，并基于系数计算一个复杂的度量值。

    参数:
    x (array-like): 离散且归一化的x序列（数值型）。
    y (array-like): 离散且归一化的y序列（数值型）。

    返回:
    float: 拟合度量值。
    """
    if count_unique(x) <= 2 or count_unique(y) <= 2:
        return 0.0
    x_std = x if np.std(x) == 1 else (x - np.mean(x)) / np.std(x) if np.std(x) > 0 else x
    y_std = y if np.std(y) == 1 else (y - np.mean(y)) / np.std(y) if np.std(y) > 0 else y
    try:
        xy1 = np.polyfit(x_std, y_std, 1)
        xy2 = np.polyfit(x_std, y_std, 2)
        return abs(2 * xy2[0]) + abs(xy2[1] - xy1[0])
    except np.RankWarning:
        return 0.0
    except Exception:
        return 0.0

def fit_error(x, y, m=2):
    """
    计算x和y之间的拟合误差。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。
    m (int): 拟合时使用的多项式的阶数。

    返回:
    float: 拟合误差。
    """
    if count_unique(x) <= m or count_unique(y) <= m:
        poly_degree = min(count_unique(x), count_unique(y)) - 1
    else:
        poly_degree = m

    if poly_degree < 1:
        return 0.0

    try:
        poly = np.polyfit(x, y, poly_degree)
        y_pred = np.polyval(poly, x)
        return np.std(y - y_pred)
    except np.RankWarning:
        return 0.0
    except Exception:
        return 0.0

def fit_noise_entropy(x, y, minc=10):
    """
    计算拟合噪声熵。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。
    minc (int): 计算熵的最小计数阈值。

    返回:
    float: 拟合噪声熵。
    """
    cx = Counter(x)
    entyx = []
    for a in cx:
        if cx[a] > minc:
            y_subset = y[x == a]
            entyx.append(discrete_entropy(y_subset))
    if len(entyx) == 0:
        return 0.0
    n = count_unique(y)
    return np.std(entyx) / np.log(n) if n > 0 else 0.0

def fit_noise_skewness(x, y, minc=8):
    """
    计算拟合噪声偏度的标准差。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。
    minc (int): 计算偏度的最小计数阈值。

    返回:
    float: 拟合噪声偏度的标准差。
    """
    cx = Counter(x)
    skewyx = []
    for a in cx:
        if cx[a] >= minc:
            y_subset = y[x == a]
            skewyx.append(normalized_skewness(y_subset))
    if len(skewyx) == 0:
        return 0.0
    return np.std(skewyx)

def fit_noise_kurtosis(x, y, minc=8):
    """
    计算拟合噪声峰度的标准差。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。
    minc (int): 计算峰度的最小计数阈值。

    返回:
    float: 拟合噪声峰度的标准差。
    """
    cx = Counter(x)
    kurtyx = []
    for a in cx:
        if cx[a] >= minc:
            y_subset = y[x == a]
            kurtyx.append(normalized_kurtosis(y_subset))
    if len(kurtyx) == 0:
        return 0.0
    return np.std(kurtyx)

def conditional_distribution_similarity(x, y, minc=12):
    """
    计算条件分布相似性。

    参数:
    x (array-like): 离散且归一化的x序列。
    y (array-like): 离散且归一化的y序列。
    minc (int): 计算条件分布的最小计数阈值。

    返回:
    float: 条件分布相似性度量。
    """
    cx = Counter(x)
    cy = Counter(y)
    yrange = sorted(cy.keys())
    ny = len(yrange)

    py = np.array([cy[i] for i in yrange], dtype=float)
    py = py / py.sum() if py.sum() > 0 else py

    pyx = []
    for a in cx:
        if cx[a] > minc:
            yx = y[x == a]
            cyx = Counter(yx)
            pyxa = np.array([cyx.get(i, 0.0) for i in yrange], dtype=float)
            if pyxa.sum() == 0:
                continue
            pyxa = pyxa / pyxa.sum()
            pyx.append(py * pyxa)  # 修正这里，将 pyx * pyxa 改为 py * pyxa

    if len(pyx) == 0:
        return 0.0

    pyx = np.array(pyx)
    pyx = pyx - pyx.mean(axis=0)
    return np.std(pyx)

In [ ]:
"""Cloud的工具函数"""
def log2(n):
    return log(n or 1, 2)

def C_MN(n: int, K: int):
    """Computes the normalizing term of NML distribution recursively. O(n+K)

    For more detail, please refer to eq (19) (Theorem1) in
    "NML Computation Algorithms for Tree-Structured Multinomial Bayesian Networks"
    https://pubmed.ncbi.nlm.nih.gov/18382603/

    and algorithm 2 in
    "Computing the Multinomial Stochastic Complexity in Sub-Linear Time"
    http://pgm08.cs.aau.dk/Papers/31_Paper.pdf


    Args
    ----------
        n (int): sample size of a dataset
        K (int): K-value multinomal distribution

    Returns
    ----------
        float: (Approximated) Multinomal Normalizing Sum

    """

    total = 1
    b = 1
    d = 10 # 10 digit precision

    #bound = int(ceil(2 + sqrt( -2 * n * np.log(2 * 10**(-d) - 100 ** (-d)))))
    bound = int(ceil(2 + sqrt(2 * n * d * log(10))))  # using equation (38)

    for k in range(1, bound + 1):
        b = (n - k + 1) / n * b
        total += b

    log_old_sum = log2(1.0)
    log_total = log2(total)
    log_n = log2(n)
    for j in range(3, K + 1):
        log_x = log_n + log_old_sum - log_total - log2(j - 2)
        x = 2 ** log_x
        log_one_plus_x = log2(1 + x)
        log_new_sum = log_total + log_one_plus_x
        log_old_sum = log_total
        log_total = log_new_sum

    if K == 1:
        log_total = log2(1.0)

    return log_total

def parametric_complexity(X, Y, model_type: str, X_ndistinct_vals=None, Y_ndistinct_vals=None):
    """Computes the Parametric Complexity of Multinomals.

    Args
    ----------
        model_type (str): ["to", "gets", "indep", "confounder"]
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
        X_ndistinct_vals (int): number of distinct values of the multinomial r.v X.
        Y_ndistinct_vals (int): number of distinct values of the multinomial r.v Y.

    Returns
    ----------
        float: Parametric Complexity of Multinomals

    """

    assert len(X)==len(Y)
    n = len(X)
    X_ndistinct_vals = X_ndistinct_vals or len(set(X))
    Y_ndistinct_vals = Y_ndistinct_vals or len(set(Y))


    if model_type == "confounder":
        return  C_MN(n=n, K=X_ndistinct_vals * Y_ndistinct_vals)

    else:
        return  C_MN(n=n, K=X_ndistinct_vals) + C_MN(n=n, K=Y_ndistinct_vals)

# ref: https://github.molgen.mpg.de/EDA/cisc/blob/master/formatter.py
def stratify(X, Y):
    """Stratifies Y based on unique values of X.
    Args:
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
    Returns:
        (dict): list of Y-values for a X-value
    """
    Y_grps = defaultdict(list)
    for i, x in enumerate(X):
        Y_grps[x].append(Y[i])
    return Y_grps

def map_to_majority(X, Y):
    """Creates a function that maps x to most frequent y.
    Args:
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
    Returns:
        (dict): map from Y-values to frequently co-occuring X-values
    """
    f = dict()
    Y_grps = stratify(X, Y)
    for x, Ys in Y_grps.items():
        frequent_y, _ = Counter(Ys).most_common(1)[0]
        f[x] = frequent_y
    return f

def update_regression(C, E, f, max_niterations=1000):
    """Update discrete regression with C as a cause variable and Y as a effect variable
    so that it maximize likelihood
    Args
    -------
        C (sequence): sequence of discrete outcomes
        E (sequence): sequence of discrete outcomes
        f (dict): map from C to Y

    """
    supp_C = list(set(C))
    supp_E = list(set(E))
    mod_E = len(supp_E)
    n = len(C)

    # N_E's log likelihood
    # optimize f to minimize N_E's log likelihood
    cur_likelihood = 0
    res = [(e - f[c]) % mod_E for c, e in zip(C, E)]
    for freq in Counter(res).values():
        cur_likelihood += freq * (log2(n) - log2(freq))

    j = 0
    minimized = True
    while j < max_niterations and minimized:
        minimized = False

        for c_to_map in supp_C:
            best_likelihood = sys.float_info.max
            best_e = None

            for cand_e in supp_E:
                if cand_e == f[c_to_map]:
                    continue

                f_ = f.copy()
                f_[c_to_map] = cand_e

                """
                if len(set(f_.values())) == 1:
                    continue
                """

                neglikelihood = 0
                res = [(e - f_[c]) % mod_E for c, e in zip(C, E)]
                for freq in Counter(res).values():
                    neglikelihood += freq * (log2(n) - log2(freq))

                if neglikelihood < best_likelihood:
                    best_likelihood = neglikelihood
                    best_e = cand_e

            if best_likelihood < cur_likelihood:
                cur_likelihood = best_likelihood
                f[c_to_map] = best_e
                minimized = True
        j += 1

    return f

def cause_effect_negloglikelihood(C, E, func):
    """Compute negative log likelihood for cause & effect pair.
    Model type : C→E

    Args
    -------
        C (sequence): sequence of discrete outcomes (Cause)
        E (sequence): sequence of discrete outcomes (Effect)
        func (dict): map from C-value to E-value

    Returns
    -------
        (float): maximum log likelihood
    """
    mod_C = len(set(C))
    mod_E = len(set(E))
    supp_C = list(set(C))
    supp_E = list(set(E))

    C_freqs = Counter(C)
    n = len(C)

    pair_cnt = defaultdict(lambda: defaultdict(int))
    for c, e in zip(C, E):
        pair_cnt[c][e] += 1

    loglikelihood = 0

    for freq in C_freqs.values():
        loglikelihood += freq * (log2(n) - log2(freq))

    for e_E in supp_E:
        freq = 0
        for e in supp_E:
            for c in supp_C:
                if (func[c] + e_E) % mod_E == e:
                    freq += pair_cnt[c][e]
        loglikelihood += freq * (log2(n) - log2(freq))

    return loglikelihood

def neg_log_likelihood(X, Y, model_type):
    """Compute negative maximum log-likelihood of the model given observations z^n.

    Args
    ------
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
        model_type (str): one of ["to", "gets", "indep", "confounder"]
        f (dict): map from Y-values to frequently co-occuring X-values
    Returns
    -----------
        (float): (negative) maximum log likelihood
    """

    n = len(X)
    loglikelihood = 0

    if model_type == "to":
        f = map_to_majority(X, Y)
        f = update_regression(X, Y, f)
        loglikelihood = cause_effect_negloglikelihood(X, Y, f)

    elif model_type == "gets":
        g = map_to_majority(Y, X)
        g = update_regression(Y, X, g)
        loglikelihood = cause_effect_negloglikelihood(Y, X, g)

    elif model_type == "indep":
        X_freqs = Counter(X)
        Y_freqs = Counter(Y)
        for freq in X_freqs.values():
            loglikelihood += freq * (log2(n) - log2(freq))
        for freq in Y_freqs.values():
            loglikelihood += freq * (log2(n) - log2(freq))

    elif model_type == "confounder":
        pair_cnt = defaultdict(lambda: defaultdict(int))
        for x, y in zip(X, Y):
            pair_cnt[x][y] += 1

        for x in list(set(X)):
            for y in list(set(Y)):
                loglikelihood += pair_cnt[x][y] * (log2(n) - log2(pair_cnt[x][y]))

    return loglikelihood

def sc(X, Y, model_type: str, X_ndistinct_vals=None, Y_ndistinct_vals=None):
    """Computes the stochastic complexity of z^n(two discrete sequences).

    Args
    ------
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
        model_type (str): ["to", "gets", "indep", "confounder"]
        X_ndistinct_vals (int): number of distinct values of the multinomial r.v X.
        Y_ndistinct_vals (int): number of distinct values of the multinomial r.v Y.

    Returns
    ----------
        float: Stochastic Complexity of a given dataset
    """
    assert len(X)==len(Y)
    X_ndistinct_vals = X_ndistinct_vals or len(set(X))
    Y_ndistinct_vals = Y_ndistinct_vals or len(set(Y))

    data_cost =  neg_log_likelihood(X, Y, model_type)
    model_cost = parametric_complexity(X, Y, model_type, X_ndistinct_vals, Y_ndistinct_vals)

    stochastic_complexity = data_cost + model_cost

    # add function code length
    if model_type == "to":
        stochastic_complexity += log2(Y_ndistinct_vals**(X_ndistinct_vals - 1) - 1)
    elif model_type == "gets":
        stochastic_complexity += log2(X_ndistinct_vals**(Y_ndistinct_vals - 1) - 1)

    return stochastic_complexity

def Cloud_print(score, llabel="X", rlabel="Y"):
    score.sort(key=lambda x: x[0])
    pred = score[0][1]
    if pred == "to":
        arrow = "⇒"
    elif pred == "gets":
        arrow = "⇐"
    elif pred == "indep":
        arrow = "⫫"
    elif pred == "confounder":
        arrow = "⇐  C ⇒"
    conf = abs(score[0][0] - score[1][0])
    out_str = "Cloud Inference Result:: %s %s %s\t Δ=%.2f" % \
                          (llabel, arrow, rlabel, conf)
    print(out_str)

def Cloud(X, Y, n_candidates=4, is_print=False, X_ndistinct_vals=None, Y_ndistinct_vals=None):
    """main function in our study.
    Cloud (Code Length-based method for Unobserved factor in Discrete data)

    Args
    ----------
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
        n_candidates (int): the number of model candidates
        is_print (bool): whether or not to print inference result
        X_ndistinct_vals (int): number of distinct values of the multinomial r.v X.
        Y_ndistinct_vals (int): number of distinct values of the multinomial r.v Y.

    Returns
    ----------
        (List) : each element is tuple that contains code length L(z^n, M) (float) and causal model' label (str)
        
    """
    if n_candidates == 4:
        MODEL_CANDIDATES = ["to", "gets", "indep", "confounder"]
    elif n_candidates == 2:
        MODEL_CANDIDATES = ["to", "gets"]
    elif n_candidates == 1:
        MODEL_CANDIDATES = ["confounder"]
    else:
        MODEL_CANDIDATES = ["to", "gets", "indep"]

    # prepare data
    le_X = LabelEncoder()
    X = le_X.fit_transform(X)
    le_Y = LabelEncoder()
    Y = le_Y.fit_transform(Y)

    results = []

    for model_type in MODEL_CANDIDATES:
        stochastic_complexity = sc(X, Y, model_type, X_ndistinct_vals, Y_ndistinct_vals)
        results.append((stochastic_complexity, model_type))

    if is_print:
        Cloud_print(results)

    return results

def Cloud_output(score):
    score.sort(key=lambda x: x[0])
    pred = score[0][1]
    conf = abs(score[0][0] - score[1][0])
    return pred, conf

In [ ]:
def Cloud_feature(dataset):
    variables = dataset.columns.drop(["X", "Y"])
    X = discretize_sequence(dataset['X'].values, ffactor=5)
    Y = discretize_sequence(dataset['Y'].values, ffactor=5)

    df = []
    for variable in variables:
        v = discretize_sequence(dataset[variable].values, ffactor=5)

        result_vX = Cloud(
            X=v, 
            Y=X,
            n_candidates=2, # select a set of model candidates
            is_print=False, # print out inferred causal direction 
            X_ndistinct_vals=11,
            Y_ndistinct_vals=11,
        )
        pred_vX, conf_vX = Cloud_output(result_vX)
        result_vY = Cloud(
            X=v, 
            Y=Y,
            n_candidates=2, # select a set of model candidates
            is_print=False, # print out inferred causal direction 
            X_ndistinct_vals=11,
            Y_ndistinct_vals=11,
        )
        pred_vY, conf_vY = Cloud_output(result_vY)
        df.append({
            "variable": variable,
            "Cloud_to(v,X)": result_vX[0][0],
            "Cloud_gets(v,X)": result_vX[1][0],
            # "Cloud_indep(v,X)": result_vX[2][0],
            # "Cloud_confounder(v,X)": result_vX[0][0],
            "Cloud_pred(v,X)": pred_vX,
            "Cloud_conf(v,X)": conf_vX,
            "Cloud_to(v,Y)": result_vY[0][0],
            "Cloud_gets(v,Y)": result_vY[1][0],
            # "Cloud_indep(v,Y)": result_vY[2][0],
            # "Cloud_confounder(v,Y)": result_vY[0][0],
            "Cloud_pred(v,Y)": pred_vY,
            "Cloud_conf(v,Y)": conf_vY,
        })

    # result_XY = Cloud(
    #     X=X, 
    #     Y=Y,
    #     n_candidates=4, # select a set of model candidates
    #     is_print=False, # print out inferred causal direction 
    #     X_ndistinct_vals=21,
    #     Y_ndistinct_vals=21,
    # )
    # pred_XY, conf_XY = Cloud_output(result_XY)

    df = pd.DataFrame(df)
    df["dataset"] = dataset.name

    # df["Cloud_to(X,Y)"] = result_XY[0][0]
    # df["Cloud_gets(X,Y)"] = result_XY[1][0]
    # df["Cloud_indep(X,Y)"] = result_XY[2][0]
    # df["Cloud_confounder(X,Y)"] = result_XY[3][0]
    # df["Cloud_pred(X,Y)"] = pred_XY
    # df["Cloud_conf(X,Y)"] = conf_XY
    
    # Reorder columns:
    df = df[["dataset"] + [colname for colname in df.columns if colname != "dataset"]]

    return df

In [ ]:
def label(adjacency_matrix):
    """
    Given a graph as adjacency_matrix, create the class labels of each variable.
    """

    adjacency_graph, adjacency_label = create_graph_label()
    labels = get_labels(adjacency_matrix, adjacency_label)
    variables = adjacency_matrix.columns.drop(["X", "Y"])

    df = pd.DataFrame({
        "variable": variables,
        "label": [labels[variable] for variable in variables],
    })
    df["dataset"] = adjacency_matrix.name

    # Reorder columns:
    df = df[["dataset"] + [colname for colname in df.columns if colname != "dataset"]]

    return df

In [ ]:
def create_some_columns(names_datasets, function):
    """
    Apply an embedding function to a list of datasets.
    """

    df = []
    for name, dataset in tqdm(names_datasets.items()):
        dataset = names_datasets[name]
        dataset.name = name
    
        try:
            df_dataset = function(dataset)
        except ValueError as e:
            print(name, e)
            raise NotImplementedError

        df_dataset["dataset"] = name
        df.append(df_dataset)

    df = pd.concat(df, axis="index").reset_index(drop=True)
    return df

In [ ]:
def create_some_columns_parallel(names_datasets, function, n_jobs=-1):
    """
    Apply an embedding function to a list of datasets.

    Parallel version.
    """

    def f(name, dataset, function):
        dataset.name = name
        df_dataset = function(dataset)
        df_dataset["dataset"] = name
        return df_dataset

    df = joblib.Parallel(n_jobs=n_jobs)(
        joblib.delayed(f)(name, dataset, function)
        for name, dataset in tqdm(names_datasets.items())
    )

    df = pd.concat(df, axis="index").reset_index(drop=True)
    return df

In [ ]:
def create_all_columns(functions_names_datasets, n_jobs=-1, create_dimension_feature = False):
    """
    given a dictionary of {function1:names, function2:names,...} apply
    the desired functions to the list of datasets and merge all of them
    in a single X_y_group dataframe.
    """

    columns = []
    if create_dimension_feature:
        dimension_feature = create_some_columns(functions_names_datasets[list(functions_names_datasets.keys())[0]], add_dimension_feature)
        columns.append(dimension_feature)
    
    for function, names_datasets in functions_names_datasets.items():
        print(f"set: {function.__name__}")

        if n_jobs != 1:
            feature_set = create_some_columns_parallel(names_datasets, function, n_jobs=n_jobs)
        else:
            feature_set = create_some_columns(names_datasets, function)

        columns.append(feature_set)

    # Merge all feature sets into a single dataframe:
    columns = functools.reduce(
        lambda left, right: pd.merge(left, right, on=["dataset", "variable"]),
        columns,
    )

    return columns

In [ ]:
X_y_group_train = pd.read_csv('mid_data/X_y_group_train_updated_v13.8_cloudXY4.csv') #需要修改为增强后数据
print(X_y_group_train.shape)
print(X_y_group_train.columns.tolist())

In [ ]:
names_datasets_train = X_train
names_graphs_train = y_train
X_y_group_train_additional = create_all_columns(
    {
        Cloud_feature: names_datasets_train,
    },
    n_jobs=-1,
)
X_y_group_train = pd.concat([X_y_group_train, X_y_group_train_additional], axis=1)
print('X_y_group_train.shape', X_y_group_train.shape)
# 去掉重复的列
X_y_group_train = X_y_group_train.loc[:,~X_y_group_train.columns.duplicated()]
print('去重后X_y_group_train.shape', X_y_group_train.shape)

In [ ]:
X_y_group_train.to_csv('./mid_data/X_y_group_train_updated_v13.9_cloudvX2vY2.csv', index=False) ### 需要修改为新的文件名，提交版本需要对应更新Cloud_feature
